In [37]:
from time import strftime
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation, metrics, decomposition, neighbors
print strftime("%Y-%m-%d %X"), "clock set"

2015-05-01 00:12:10 clock set


In [38]:
# load X_given and y_given
with open('../data/train.csv') as f:
    given_csv = f.readlines()
    # number of columns in each row is 1 + 28^2 = 1 + 784
    given_csv = [np.array(map(int, x.split(','))) for x in given_csv[1:]]
    X_given = [x[1:] for x in given_csv]
    y_given = [x[0] for x in given_csv]
print strftime("%Y-%m-%d %X"), "given data loaded"

# load X_test
with open('../data/test.csv') as f:
    test_csv = f.readlines()
    # number of columns in each row is 28^2 = 784
    X_test = [np.array(map(int, x.split(','))) for x in test_csv[1:]]
print strftime("%Y-%m-%d %X"), "testing data loaded"

print len(X_given)

2015-05-01 00:12:47 given data loaded
2015-05-01 00:13:10 testing data loaded
42000


In [39]:
# X_given = X_given[0:20000] # reduce data for tests
# y_given = y_given[0:20000]

X_train, X_validate, y_train, y_validate = cross_validation.train_test_split(X_given, y_given, test_size=0, random_state=0)
print strftime("%Y-%m-%d %X"), "cross-validation loaded"

2015-05-01 00:13:10 cross-validation loaded


In [40]:
X_train_L = [np.reshape([list(row[1:28]) + [0] for row in np.reshape(i, [28, 28])], 28*28) for i in X_train]
print strftime("%Y-%m-%d %X"), "computed shift left data"
X_train_R = [np.reshape([[0] + list(row[0:27]) for row in np.reshape(i, [28, 28])], 28*28) for i in X_train]
print strftime("%Y-%m-%d %X"), "computed shift right data"


2015-05-01 00:13:25 computed shift left data
2015-05-01 00:13:39 computed shift right data


In [41]:
X_train_U = [np.reshape(np.concatenate([np.reshape(i, [28, 28])[1:28], [np.array([0] * 28)]]), -1) for i in X_train]
print strftime("%Y-%m-%d %X"), "computed shift up data"
X_train_D = [np.reshape(np.concatenate([[np.array([0] * 28)], np.reshape(i, [28, 28])[0:27]]), -1) for i in X_train]
print strftime("%Y-%m-%d %X"), "computed shift down data"

2015-05-01 00:13:41 computed shift up data
2015-05-01 00:13:43 computed shift down data


In [42]:
# two to the left and two to the right
X_train_LL = [np.reshape([list(row[1:28]) + [0] for row in np.reshape(i, [28, 28])], 28*28) for i in X_train_L]
print strftime("%Y-%m-%d %X"), "computed shift left twice data"
X_train_RR = [np.reshape([[0] + list(row[0:27]) for row in np.reshape(i, [28, 28])], 28*28) for i in X_train_R]
print strftime("%Y-%m-%d %X"), "computed shift right twice data"

2015-05-01 00:13:59 computed shift left twice data
2015-05-01 00:14:14 computed shift right twice data


In [43]:
X_train_UU = [np.reshape(np.concatenate([np.reshape(i, [28, 28])[1:28], [np.array([0] * 28)]]), -1) for i in X_train_U]
print strftime("%Y-%m-%d %X"), "computed shift up twice data"
X_train_DD = [np.reshape(np.concatenate([[np.array([0] * 28)], np.reshape(i, [28, 28])[0:27]]), -1) for i in X_train_D]
print strftime("%Y-%m-%d %X"), "computed shift down twice data"

2015-05-01 00:14:16 computed shift up twice data
2015-05-01 00:14:19 computed shift down twice data


In [44]:
X_train_expanded = X_train + X_train_L + X_train_R + X_train_U + X_train_D + X_train_LL + X_train_RR + X_train_UU + X_train_DD
y_train_expanded = y_train * 9
print strftime("%Y-%m-%d %X"), "added data"

2015-05-01 00:14:19 added data


In [45]:
len(X_train), len(X_train_expanded), len(y_train_expanded)

(42000, 378000, 378000)

In [49]:
pca65 = decomposition.RandomizedPCA(n_components=65, whiten=False)
X_transform65 = pca65.fit_transform(X_train_expanded)
print strftime("%Y-%m-%d %X"), "PCA finished"
knn1 = neighbors.KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_transform65, y_train_expanded)
print strftime("%Y-%m-%d %X"), "knn finished"
# z = knn1.predict(pca65.transform(X_validate))
# print metrics.accuracy_score(y_validate, z)
# print strftime("%Y-%m-%d %X"), "finished scoring"

2015-05-01 05:20:55 PCA finished
2015-05-01 05:21:01 knn finished


In [50]:
z_test = knn1.predict(pca65.transform(X_test))
print strftime("%Y-%m-%d %X"), "predicted test data"


2015-05-01 05:29:48 predicted test data


In [51]:
with open('pca_knn_with_translation_plus_shape_65.csv', 'w') as f:
    f.write('ImageId,Label\n')
    for i in xrange(28000):
        f.write(str(i+1) + ',' + str(z_test[i]) + '\n')
